In [80]:
from ipywidgets import HBox, VBox, IntSlider, interactive, SelectionSlider, Label, Layout, HTML, Output, GridBox
from IPython.display import display
import folium
import json
import pandas as pd
import geopandas as gpd
import geopatra
import random
from branca.colormap import linear

In [81]:
colormap = linear.OrRd_09.scale(0, 400000)

In [82]:
def test(passenger_count,private_bus_to_public_bus,travel_risk,confirmed_risk,qtn_risk):
    
    display(
        Label(value='', layout = Layout(height="50"))
    )
    
    district_data_df = pd.read_csv("dummy_assam.csv")
    numbers_df = district_data_df
    district_data_df["AtRisk"] = (
        numbers_df["BusCount"] 
        * passenger_count
        * private_bus_to_public_bus
        / passenger_count
        * private_bus_to_public_bus
        * travel_risk
        + numbers_df["QTNCount"] / qtn_risk
        + confirmed_risk * numbers_df["CaseCount"]
    ).astype(int)
    
    present = pd.DataFrame(
        data = {
            'District':district_data_df['district'].tolist(),
            'AtRisk':district_data_df['AtRisk'].tolist()
        }
    )
    geo_df = gpd.read_file("assam.json")
    df = gpd.GeoDataFrame(pd.merge(district_data_df, geo_df, on="district"), geometry='geometry')
    
    map = df.folium.chloropeth(width=1200, height= 600,color_by="AtRisk", color=colormap, style = {'color': 'black',
                    'weight': 1.5,
                    'dashArray': '10, 5',
                    'fillOpacity': 0.9,
                  }, zoom=7, tiles='google', tooltip=['district','AtRisk'], location=(26.244156, 92.537842))
    
    widget1 = Output()
    widget2 = Output()
    
    with widget1:
        display(present)
    
    with widget2:
        display(map)
        
    hbox = GridBox([widget1, widget2], layout=Layout(grid_template_columns='300px 1000px'))
    
    display(hbox)

In [83]:
slider_layout = Layout(width='500px',height='20px')
style_dict = {'description_width': '150px'}
float_value = [10**-i for i in range(-2,3)]
float_selections = [("%g"%i,i) for i in float_value]


a = interactive(
    test,
    passenger_count = IntSlider(
        value=50, 
        min=1, 
        max=100,
        description="Passenger Count: ",
        style = style_dict,
        layout = slider_layout
    ),
    
    private_bus_to_public_bus = IntSlider(
        value=50, 
        min=1, 
        max=100,
        description="Private bus to Public bus: ",
        style = style_dict,
        layout = slider_layout
    ),
    
    confirmed_risk = IntSlider(
        value=50, 
        min=1, 
        max=100,
        description="Confirmed Risk: ",
        style = style_dict,
        layout = slider_layout
    ),
    
    travel_risk = SelectionSlider(
        options=float_selections,
        value=1,
        description="Travel Risk: ",
        style = style_dict,
        layout = slider_layout
    ),
    
    qtn_risk = SelectionSlider(
        options=float_selections,
        value=1,
        description="QTN Risk: ",
        style = style_dict,
        layout = slider_layout
    ),
)

In [84]:
display(a)

interactive(children=(IntSlider(value=50, description='Passenger Count: ', layout=Layout(height='20px', width=…